In [1]:
import numpy as np
from math import sin, cos, atan2, sqrt

In [25]:
value = np.loadtxt('./moje_dane.txt')
value = value*1000
param = ['sh','year','month','day','hour', 'minuts', 'sec', 'x', 'u', 'd2x', 'y', 'v', 'd2y', 'z', 'w', 'd2z']
rinex = dict(zip(param,value))

In [27]:
# stale
GM = 398600.4418e+9
ae = 6378136
C20 = -1082625.7e-9
omega = 7.292115e-5


#dane poczatkowe
x = rinex['x']
y = rinex['y']
z = rinex['z']

u = rinex['u']
v = rinex['v']
w = rinex['w']

d2x = rinex['d2x']
d2y = rinex['d2y']
d2z = rinex['d2z']

tsv = 12*3600+41*60+36
tb = rinex['hour']/1000*3600+rinex['minuts']/1000*60 + rinex['sec']/1000
tb_0 = tb
t = tsv

roz = t - tb

ilosc = int(abs(roz/51))
def func_r(x,y,z):
    return sqrt(x**2+y**2+z**2)


print(tb, u,v,w,round(x,4),round(y,4), round(z,4))
h = -51
for i in range(1,ilosc+1):
    #     if i == ilosc+1:
    #         h = roz+ilosc*60
    #     #print(u,v,w,x,y,z)
    r = func_r(x,y,z)
    K1, K2, K3, K4, K5, K6 = [0 for x in  range(6)]
    k = []
    for m in range(4):
        if m != 3:
            xt = x + K4/2
            yt = y + K5/2
            zt = z + K6/2
            ut = u + K1/2
            vt = v + K2/2
            wt = w + K3/2
        else:
            xt = x + K4
            yt = y + K5
            zt = z + K6
            ut = u + K1
            vt = v + K2
            wt = w + K3
        r = func_r(xt,yt,zt)
        
        f1 = (-GM/r**3)*xt+3/2*C20*((GM*ae**2)/r**5)*xt*(1-((5*zt**2)/r**2))+omega**2*xt+2*omega*vt+d2x
        f2 = (-GM/r**3)*yt+3/2*C20*((GM*ae**2)/r**5)*yt*(1-((5*zt**2)/r**2))+omega**2*yt-2*omega*ut+d2y
        f3 = (-GM/r**3)*zt+3/2*C20*((GM*ae**2)/r**5)*zt*(3-((5*zt**2)/r**2))+d2z
        
        K1, K2, K3, K4, K5, K6 =  h * f1, h * f2, h * f3, h * ut, h * vt, h * wt
        K = np.array([K1, K2, K3, K4, K5, K6])
        k.append(K)
    k = np.transpose(np.array(k))
    u = u + 1 / 6 * (k[0, 0] + 2 * k[0, 1] + 2 * k[0, 2] + k[0, 3])
    v = v + 1 / 6 * (k[1, 0] + 2 * k[1, 1] + 2 * k[1, 2] + k[1, 3])
    w = w + 1 / 6 * (k[2, 0] + 2 * k[2, 1] + 2 * k[2, 2] + k[2, 3])
    x = x + 1 / 6 * (k[3, 0] + 2 * k[3, 1] + 2 * k[3, 2] + k[3, 3])
    y = y + 1 / 6 * (k[4, 0] + 2 * k[4, 1] + 2 * k[4, 2] + k[4, 3])
    z = z + 1 / 6 * (k[5, 0] + 2 * k[5, 1] + 2 * k[5, 2] + k[5, 3])
    tb += h
    print(tb, u,v,w,round(x,4),round(y,4), round(z,4))
print(', '.join([str(int(rinex['sh']/1000)), str(int(tb_0)), str(t), str(round(x,3)),str(round(y,3)), str(round(z,3))]))

45900.0 -2730.002403259 -286.3874435425 1781.465530396 12014235.8398 10254595.7031 20038555.1758
45849.0 -2716.3260656277935 -296.86991515602153 1805.931665888475 12153118.1089 10269469.258 19947076.0709
45798.0 -2702.440358201412 -307.23554014654144 1830.2849477579455 12291297.5392 10284874.4481 19854352.0644
45747.0 -2688.346832976215 -317.4822628371828 1854.5238593019208 12428763.4925 10300805.2617 19760388.9505
45696.0 -2674.0470672145816 -327.60804497156437 1878.6468909542396 12565505.41 10317255.5829 19665192.6007
21, 45900, 45696, 12565505.41, 10317255.583, 19665192.601
